# 04 - Testing
---

### **Introduction**
Software testing is the practice of systematically checking that code behaves as expected and correctly implements its intended functionality. It involves writing tests that verify individual components (unit tests), interactions between components (integration tests), and overall system behaviour, helping to detect bugs early and prevent regressions. Good testing improves reliability, maintainability, and confidence in code changes.

### **Unit Tests**
Unit testing is the practice of testing individual components of a program, (usually functions) in isolation to ensure that they behave as intended. Each unit test targets a small, well-defined piece of logic and verifies that, given certain inputs, it produces the expected outputs. By keeping tests focused and independent, unit testing makes it easier to detect bugs early, simplify debugging, and ensure that future changes do not break existing functionality.

We typically use the `pytest` package to implement unit tests. The typical approach is to designate a `tests` directory in the root of the project with separate testing scripts for different sections of the codebase. Suppose we wrote the following python file as part of our project which contains some helpful functions for performing certain arithmetic operations. 

In [2]:
# Script to test
def divide(a, b):
    """Return the division of two numbers. Raises ZeroDivisionError if b is zero."""
    if b == 0:
        raise ZeroDivisionError("Cannot divide by zero")
    return a / b


def divide_then_square(a, b):
    a_divided_by_b = divide(a, b)
    return a_divided_by_b ** 2

We would then create a script in our `tests` directory for testing the code.  Inside this script we would write the tests in the form of functions which call the function we would like to test and check if the value is what we expect. We can use the `assert` python keyword to check if two values are the same. Note that all the test function names must start with the word test for `pytest` to recognise them as unit tests. A typical convention is that the test for a function called say `my_function` is named `test_my_function`. This makes it explicit what exactly the test is for. 

Note that to run the tests, we don't run the test script itself. Instead we run the terminal command `pytest`. This will run all the tests and display whether each one passes or fails. If it fails you will see the stack trace to help with debugging. 

In [ ]:
import pytest

def test_divide():
    assert divide(6, 2) == 3
    assert divide(-6, -2) == 3
    assert divide(5, 2) == 2.5

    with pytest.raises(ZeroDivisionError):
        divide(1, 0)

### **Parametrising Unit Tests**
In the above example we explicitly made the function calls to `divide` inside our test function. The disadvantage of doing this is that pytest will only treat it as a single test so if it fails it will be unclear which of the tested scenarios failed and which succeeded. To fix this, we can parametrise unit tests by specifying series of input parameters to test as different scenarios. For parametrised unit tests, pytest treats each input as a separate test, allowing us to easily see which scenarios succeed and which fail. 

You can parametrise tests using the `pytest.mark.parametrize decorator` which takes exactly two arguments:
- A string which matches the argument names you would like to parametrise. Be careful with the syntax here; it should be "arg1, arg2, arg3", not "arg1", "arg2", "arg3"
- A list where each element is a tuple whose length is the number of parametrised arguments. Ensure the order matches the order of arguments specifies in the string

For parametrised unit tests, pytest takes each tuple in the list and maps each value to an argument name in the function based on the string. It then runs the test using those parameters. Be aware that pytest runs the tests in the reverse order to which they are specified - this can cause confusion when debugging why tests have failed. 


In [4]:
from unittest.mock import patch

@pytest.mark.parametrize("a, b, expected", [
    (6, 2, 3),
    (-6, -2, 3),
    (5, 2, 2.5),
    (9, 3, 3)
])
def test_divide(a, b, expected):
    assert divide(a, b) == expected


@pytest.mark.parametrize("a, b", [
    (1, 0),
    (10, 0),
    (-5, 0)
])
def test_divide_zero_division(a, b):
    with pytest.raises(ZeroDivisionError):
        divide(a, b)

### **Monkey Patching**
Suppose you wanted to test a function that depends on another function. For example `divide_then_square` which depends on `divide`. We need to isolate the functionality of `divide_then_square` from `divide` so that a problem with `divide` does not affect the test for `divide_then_square`. To do this we use the patch decorator to mock/overwrite the value of a variable or function call inside the test. In this example we use the patch decorator to mock the return value of the `divide` function. This is also included as the first argument. We pass the value we would like to mock with as an argument in the parametrize call. Inside the test, we overwrite the return value of the `divide` function by setting its `.return_value` to the value we supplied as an argument.

This can be a useful approach if for instance you function calls data from an external service such as an api. 


In [ ]:

@pytest.mark.parametrize("a, b, mock_divide_value, expected", [
    (2, 1, 2, 4),
    (10, 2, 5, 25),
])
@patch(__name__ + ".divide")
def test_divide_then_square(mock_divide, a, b, mock_divide_value, expected):
    mock_divide.return_value = mock_divide_value
    result = divide_then_square(a, b)
    assert result == expected